In [ ]:
import yaml
config = yaml.safe_load(open("../config/datasets.yml", "r"))

In [ ]:
DATASET = "AbbySlidingWindow"
DATASET_DIR = config[DATASET]["path"]
print(DATASET_DIR)

In [ ]:
from fish_benchmark.utils import get_files_of_type
import numpy as np
annotation_files = get_files_of_type(DATASET_DIR, ".txt")

annotations = []
for file in annotation_files:
    annotations.append(np.loadtxt(file, delimiter="\t"))

annotations = np.concatenate(annotations, axis=0)
annotations.shape

In [ ]:
annotations.sum(axis=0)

In [ ]:
from fish_benchmark.data.dataset import AbbyDataset
from fish_benchmark.models import get_input_transform

In [ ]:
from tqdm import tqdm
for frame, label in tqdm(dataset):
    pass

In [7]:
input_transform = get_input_transform("videomae")
dataset = AbbyDataset(
        DATASET_DIR, 
        train=True, 
        transform=input_transform, 
        label_type='onehot', 
        window_size=16, 
        tolerance_region = 7,
        samples_per_window = 16,
        step_size=1, 
        is_image_dataset=False
)

frame, label = next(iter(dataset))
from fish_benchmark.debug import serialized_size
print(serialized_size(frame))

1021183132


In [ ]:
from tqdm import tqdm
import torch


labels = []
for frame, label in tqdm(dataset):
    labels.append(label)

labels = torch.stack(labels)
print(labels.sum(axis=0)/labels.shape[0])

106it [00:06, 15.40it/s]


KeyboardInterrupt: 

In [3]:
import time
import torch 
from fish_benchmark.debug import serialized_size
x = torch.randn(16, 3, 224, 224)
start = time.time()
torch.save(x, "/share/j_sun/jth264/test.pt")
print("Time:", time.time() - start)

Time: 0.017487525939941406


In [4]:
clips = [torch.randn(16, 3, 224, 224) for _ in range(100)]
stacked = torch.stack(clips)
clip0 = stacked[0]

print(clip0.storage().size())          # Very large! Entire tensor storage
print(serialized_size(clip0))          # Very large! ~1GB, same as full tensor
print(clip0.clone().storage().size())  # Just right! ~9MB worth

240844800
963380380
2408448
